In [1]:
import pandas as pd

data = pd.read_csv('data.csv', 
                   encoding='unicode_escape',
                   parse_dates = ['InvoiceDate'], 
                   dtype = {'CustomerID': str,
                          'InvoiceNo': str})
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [2]:
def check_data(dataframe):
    print(' shape '.center(55, '-'))
    print(' rows: {}' .format(dataframe.shape[0]))
    print(' columns: {}' .format(dataframe.shape[1]))
    print(' types '.center(55, '-'))
    print(dataframe.dtypes)
    print(' missing values '.center(55, '-'))
    print(dataframe.isnull().sum())
    print(' duplicated values '.center(55, '-'))
    print(dataframe.duplicated().sum())
    print(' quantiles '.center(55, '-'))
    print(dataframe.quantile([0, 0.05, 0.5, 0.95, 0.99, 1]).T)

check_data(data)


------------------------ shape ------------------------
 rows: 541909
 columns: 8
------------------------ types ------------------------
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object
-------------------- missing values -------------------
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
------------------ duplicated values ------------------
5268
---------------------- quantiles ----------------------
               0.00  0.05  0.50   0.95   0.99     1.00
Quantity  -80995.00  1.00  3.00  29.00  100.0  80995.0
UnitPrice -11062.06  0.42  2.08   9.95   18.0  38970.0


In [3]:
data.describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


##### 初步探索結果
##### 1. Description跟CustomerID有缺失值
##### 2. Quantity跟UnitPrice有極端值
##### 3. 負數表示有退貨訂單

## Data Preprocessing

In [4]:
def replace_with_thresholds(dataframe, variable, q1 = 0.25, q3 = 0.75):
    df = dataframe.copy()
    quartile1 = df[variable].quantile(q1)
    quartile3 = df[variable].quantile(q3)
    iqr = quartile3 - quartile1
    up_limit = quartile3 + 1.5*iqr
    low_limit = quartile1 - 1.5*iqr
    df.loc[(df[variable] < low_limit), variable] = low_limit
    df.loc[(df[variable] > up_limit), variable] = up_limit

    return df

def preprocess(dataframe):
    df = dataframe.copy()
    # remove missing values
    df = df.dropna()
    # remove cancelled orders
    df = df[~df['InvoiceNo'].str.contains('C', na = False)]
    df = df[df['Quantity'] > 0]
    # replacing outliers
    df = replace_with_thresholds(df, 'Quantity', q1 = 0.01, q3 = 0.99)
    df = replace_with_thresholds(df, 'UnitPrice', q1 = 0.01, q3 = 0.99)
    # total price
    df['TotalPrice'] = df['Quantity']*df['UnitPrice']

    return df

df = preprocess(data)


In [5]:
# remove the records that UnitPrice = 0
df = df[df['UnitPrice'] != 0]

In [6]:
df.describe()

,Quantity,UnitPrice,TotalPrice
count,397884.000000,397884.000000,397884.000000
mean,11.830797,2.893454,20.629824
std,25.523078,3.227090,51.828592
min,1.000000,0.001000,0.001000
25%,2.000000,1.250000,4.680000
50%,6.000000,1.950000,11.800000
75%,12.000000,3.750000,19.800000
max,298.500000,37.060000,3268.575000


## RFM Analysis

In [7]:
print(df['InvoiceDate'].max())

2011-12-09 12:50:00


In [8]:
import datetime as dt
today = dt.datetime(2011, 12, 11)

In [9]:
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate' : lambda x :(today - x.max()).days,
    'InvoiceNo' : lambda x : x.nunique(),
    'TotalPrice' : lambda x : x.sum()
})

rfm = rfm.rename(columns={'InvoiceDate' : 'Recency', 'InvoiceNo' : 'Frequency', 'TotalPrice' : 'Monetary'})
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346,326,1,310.44
12347,3,7,4310.00
12348,76,4,1770.78
12349,19,1,1491.72
12350,311,1,331.46


In [10]:
rfm.describe()

,Recency,Frequency,Monetary
count,4338.000000,4338.000000,4338.000000
mean,93.059474,4.272015,1892.180055
std,100.012264,7.697998,7706.207355
min,1.000000,1.000000,3.750000
25%,18.000000,1.000000,303.307500
50%,51.000000,2.000000,663.100000
75%,142.750000,5.000000,1631.107500
max,374.000000,209.000000,266163.525000


## Assigning RFM Scores

In [11]:
rfm['recency_score'] = pd.qcut(rfm['Recency'], 2, [2, 1])
rfm['frequency_score'] = pd.qcut(rfm['Frequency'].rank(method="first"), 2, [1, 2])
rfm['monetary_score'] = pd.qcut(rfm['Monetary'], 2, [1, 2])
rfm['RFM_score'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str)
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_score
CustomerID,,,,,,,
12346,326,1,310.44,1,1,1,111
12347,3,7,4310.00,2,2,2,222
12348,76,4,1770.78,1,2,2,122
12349,19,1,1491.72,2,1,2,212
12350,311,1,331.46,1,1,1,111


In [19]:
rfm.reset_index(inplace=True)
rfm

,CustomerID,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_score,Segment
0,12346,326,1,310.44,1,1,1,111,一般挽留客戶
1,12347,3,7,4310.00,2,2,2,222,重要價值客戶
2,12348,76,4,1770.78,1,2,2,122,重要保持客戶
3,12349,19,1,1491.72,2,1,2,212,重要發展客戶
4,12350,311,1,331.46,1,1,1,111,一般挽留客戶
...,...,...,...,...,...,...,...,...,...
4333,18280,278,1,180.60,1,1,1,111,一般挽留客戶
4334,18281,181,1,80.82,1,1,1,111,一般挽留客戶
4335,18282,8,2,178.05,2,2,1,221,一般價值客戶
4336,18283,4,16,2094.88,2,2,2,222,重要價值客戶


In [13]:
def segment(score):
    if score == '222':
        return '重要價值客戶'
    elif score == '212':
        return '重要發展客戶'
    elif score == '122':
        return '重要保持客戶'
    elif score == '112':
        return '重要挽留客戶'
    elif score == '221':
        return '一般價值客戶'
    elif score == '211':
        return '一般發展客戶'
    elif score == '121':
        return '一般保持客戶'
    elif score == '111':
        return '一般挽留客戶'
rfm['Segment'] = rfm['RFM_score'].apply(segment)
    

In [21]:
result = rfm.groupby('Segment').agg({'CustomerID':'count'}).reset_index()
result

,Segment,CustomerID
0,一般保持客戶,198
1,一般價值客戶,198
2,一般挽留客戶,1252
3,一般發展客戶,521
4,重要保持客戶,452
5,重要價值客戶,1321
6,重要挽留客戶,260
7,重要發展客戶,136
